In [ ]:
import pydicom as dicom
import matplotlib.pylab as plt

# specify your image path
image_path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00185/T2w/Image-40.dcm'
ds = dicom.dcmread(image_path)

plt.imshow(ds.pixel_array)

In [ ]:
import numpy as np
import pydicom
def load_dicom(path):
    
    data = pydicom.dcmread(path)
    '''
    Returns the image data as a numpy array.
    '''  
    if np.max(data.pixel_array)==0:
        img = data.pixel_array
    else:
        img = data.pixel_array/np.max(data.pixel_array)
        img = (img * 255).astype(np.uint8)
        
    return img


In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_csv('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
data = df.set_index("BraTS21ID")
data = data.drop([109, 123, 709], axis=0)
df=data.reset_index()

In [ ]:
import numpy as np
labels=np.array(df['MGMT_value'])

**Verisetindeki görüntüleri inceleme**

In [ ]:
from glob import glob
imagePatches = glob('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*/', recursive=True)

In [ ]:
import os
files=[]
files.append ('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00109/')
files.append ('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00709/')
files.append ('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00123/')
for file in files:
    imagePatches.remove(file)

In [ ]:
len(imagePatches)

**READ ALL SEQUENCES FOR EACH SUBFOLDER !**

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
imagePatches.sort(key=natural_keys)
flair_patches = []
t1w_patches=[]
t1wce_patches=[]
t2w_patches=[]
for subfolder in imagePatches:
    flair_patches.append(glob(subfolder + 'FLAIR/**/**/*.dcm', recursive=True))
    t1w_patches.append(glob(subfolder + 'T1w/**/**/*.dcm', recursive=True))
    t1wce_patches.append(glob(subfolder + 'T1wCE/**/**/*.dcm', recursive=True))
    t2w_patches.append(glob(subfolder + 'T2w/**/**/*.dcm', recursive=True))

In [ ]:
def all_slice(sequence, list_name):
    for x in range(0,len(sequence)):
        sequence[x].sort(key=natural_keys)
        list_name.append((sequence[x][0:len(sequence[x])]))
    return list_name

In [ ]:
all_flair_patches = []
all_slice(flair_patches,all_flair_patches)
all_t1w_patches =[]
all_t1w_patches = all_slice(t1w_patches,all_t1w_patches)
all_t1wce_patches=[]
all_t1wce_patches = all_slice(t1wce_patches,all_t1wce_patches)
all_t2w_patches=[]
all_t2w_patches = all_slice(t2w_patches,all_t2w_patches)

**For each patch, take middle 3 slices as an input.**

In [ ]:
import cv2
def create_input(patches):
    inputs=np.zeros((582,256,256,3))
    for i in range(0,len(patches)):
        for j in range(len(patches[i])//2,(len(patches[i])//2)+3):
            img= load_dicom(patches[i][j])
            image_array = cv2.resize(img, (256,256), interpolation=cv2.INTER_AREA)
            image_array = np.expand_dims(image_array, -1)
        inputs[i]=image_array
    return inputs

In [ ]:
t2w_inputs = create_input(all_t2w_patches)
flair_inputs = create_input(all_flair_patches)
t1wce_inputs = create_input(all_t1wce_patches)
t1w_inputs = create_input(all_t1w_patches)

In [ ]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import keras

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation, GlobalAveragePooling2D
from keras.optimizers import Adam, Adamax, Adagrad
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

In [ ]:
flair_inputs= np.asarray(flair_inputs)
t1w_inputs= np.asarray(t1w_inputs)
t1wce_inputs= np.asarray(t1wce_inputs)
t2w_inputs = np.asarray(t2w_inputs)

In [ ]:
from sklearn.model_selection import train_test_split
from IPython.display import Image
from sklearn.metrics import confusion_matrix
np.random.seed(1)

In [ ]:
from tensorflow.keras import layers
def get_model(optimizer):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((256,256,3,1))
    
    

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu",padding='same')(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    #x = layers.Conv3D(filters=64, kernel_size=3, activation="relu",padding='same')(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    #x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu",padding='same')(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    #x = layers.Conv3D(filters=256, kernel_size=3, activation="relu",padding='same')(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    #x = layers.BatchNormalization()(x)
    
    x = layers.GlobalAveragePooling3D()(x)
    #x=layers.Flatten()(x)
    #x = layers.Dense(units=1024, activation="relu")(x)
    #x = layers.Dropout(0.3)(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])
    return model


# Build model.
model = get_model('Adam')

**For each input set, train the 3D-CNN model and save them.**

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, mode='auto', verbose = 1)
callbacks = [ModelCheckpoint(filepath='/kaggle/output/models/best_model.h5', monitor='val_loss', save_best_only=True)]
x_train, x_valid, y_train, y_valid = train_test_split(flair_inputs, labels, test_size = 0.2, random_state = 1)
optimizers= ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=3, mode='min')
x_train = x_train/255
x_valid = x_valid/255
model=get_model('Adam')
history = model.fit(x_train, y_train, epochs=50, batch_size=8,shuffle=True, validation_data=(x_valid, y_valid),callbacks=[early_stop])

In [ ]:
model.save('/kaggle/t1winputs.hdf5')
model_t1w= keras.models.load_model('/kaggle/t1winputs.hdf5')

In [ ]:
model.save('/kaggle/t1wceinputs.hdf5')
model_t1wce= keras.models.load_model('/kaggle/t1wceinputs.hdf5')

In [ ]:
model.save('/kaggle/t2winputs.hdf5')
model_t2w= keras.models.load_model('/kaggle/t2winputs.hdf5')

In [ ]:
model.save('/kaggle/flairinputs.hdf5')
model_flair= keras.models.load_model('/kaggle/flairinputs.hdf5')

In [ ]:
preds_flair=model_flair.predict(x_valid)
preds_t1w=model_t1w.predict(x_valid)
preds_t2w=model_t2w.predict(x_valid)
preds_t1wce=model_t1wce.predict(x_valid)

In [ ]:
mean = [(g + h + a + b) / 4 for g, h, a, b in zip(preds_flair,preds_t1w, preds_t2w,preds_t1wce)]

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score

fpr , tpr , thresholds = roc_curve ( y_valid , mean)

In [ ]:
def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  
plot_roc_curve (fpr,tpr) 

In [ ]:
auc_score=roc_auc_score(y_valid,mean)
print(auc_score)

**TESTING PART !!!**

**Load my models**

In [ ]:
model_t1w= keras.models.load_model('/kaggle/t1winputs.hdf5')
model_t1wce= keras.models.load_model('/kaggle/t1wceinputs.hdf5')
model_t2w= keras.models.load_model('/kaggle/t2winputs.hdf5')
model_flair= keras.models.load_model('/kaggle/flairinputs.hdf5')

In [ ]:
import cv2
def create_input(patches, number_image):
    inputs=np.zeros((number_image,256,256,3))
    for i in range(0,len(patches)):
        for j in range(len(patches[i])//2,(len(patches[i])//2)+3):
            img= load_dicom(patches[i][j])
            image_array = cv2.resize(img, (256,256), interpolation=cv2.INTER_AREA)
            image_array = np.expand_dims(image_array, -1)
        inputs[i]=image_array
    return inputs

In [ ]:
testimages= glob('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*/', recursive=True)
testimages.sort(key=natural_keys)
flair_patches = []
t1w_patches=[]
t1wce_patches=[]
t2w_patches=[]
for subfolder in testimages:
    flair_patches.append(glob(subfolder + 'FLAIR/**/**/*.dcm', recursive=True))
    t1w_patches.append(glob(subfolder + 'T1w/**/**/*.dcm', recursive=True))
    t1wce_patches.append(glob(subfolder + 'T1wCE/**/**/*.dcm', recursive=True))
    t2w_patches.append(glob(subfolder + 'T2w/**/**/*.dcm', recursive=True))
all_flair_patches = []
all_slice(flair_patches,all_flair_patches)
all_t1w_patches =[]
all_t1w_patches = all_slice(t1w_patches,all_t1w_patches)
all_t1wce_patches=[]
all_t1wce_patches = all_slice(t1wce_patches,all_t1wce_patches)
all_t2w_patches=[]
all_t2w_patches = all_slice(t2w_patches,all_t2w_patches)
t2w_inputs = create_input(all_t2w_patches, len(testimages))
flair_inputs = create_input(all_flair_patches,len(testimages))
t1wce_inputs = create_input(all_t1wce_patches,len(testimages))
t1w_inputs = create_input(all_t1w_patches,len(testimages))
testflair_inputs= np.asarray(flair_inputs)/255
testt1w_inputs= np.asarray(t1w_inputs)/255
testt1wce_inputs= np.asarray(t1wce_inputs)/255
testt2w_inputs = np.asarray(t2w_inputs)/255
preds_flair=model_flair.predict(testflair_inputs)
preds_t1w=model_t1w.predict(testt1w_inputs)
preds_t2w=model_t2w.predict(testt2w_inputs)
preds_t1wce=model_t1wce.predict(testt1wce_inputs)
mean = [(g + h + a + b) / 4 for g, h, a, b in zip(preds_flair,preds_t1w, preds_t2w,preds_t1wce)]

In [ ]:

sample=pd.read_csv("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv",index_col="BraTS21ID")
sample["MGMT_value"] = 0
sample["MGMT_value"] = [str(a)[1:-1] for a in mean]
sample["MGMT_value"].to_csv("submission.csv")


In [ ]:
sample